In [1]:
import os
import re
from datetime import datetime
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
from dateutil.relativedelta import relativedelta


regions = ['Cherkasy', 'Chernihiv', 'Chernivtsi', 'Crimea', "Dnipropetrovs'k", "Donets'k", "Ivano-Frankivs'k", 'Kharkiv', 'Kherson', "Khmel'nyts'kyy", 'Kiev', 'Kie', 'Kirovohrad', "Luhans'k", "L'viv", 'Mykolayiv', 'Odessa', 'Poltava', 'Rivne', "Sevastopol'", 'Sumy', "Ternopil'", 'Transcarpathia', 'Vinnytsya', 'Volyn', 'Zaporizhzhya', 'Zhytomyr',]


def df_relativedelta(week):
    return relativedelta(weeks=+week)
    
    
def df_datetime_date(year):
    return datetime.date(year, 1, 1)


def change_data_format(df):
    df.Year = df.Year.astype('int64')
    df.Week = df.Week.astype('int64')
    df['date'] = df.Year.apply(df_datetime_date) + df.Week.apply(df_relativedelta)
    df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
    return df


def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result


def create_dataframe(path):
    concatenate_files=[]
    for filename in os.listdir(path):
        f = os.path.join(path, filename)
        if os.path.isfile(f): 
            headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'area']
            df = pd.read_csv(f, header = 1, names = headers)
            df['Year']=df['Year'].apply(lambda cw : remove_tags(cw))
            df = df.drop(df.loc[df['VHI'] == -1].index)
            df = df[df['Week'].notna()]
            df['area'] = regions[int(f.split('_')[1])-1]
            concatenate_files.append(df)
    return pd.concat(concatenate_files, axis=0, ignore_index=True)

In [ ]:
from spyre import server
import numpy as np
from matplotlib import pyplot as plt
server.include_df_index = True


class NOAA(server.App):
    inputs = [{
                "type": 'dropdown',
                "label": 'NOAA',
                "options": [
                    {"label": "VCI", "value": "VCI"},
                    {"label": "TCI", "value": "TCI"},
                    {"label": "VHI", "value": "VHI"}],
                    "key": 'ticker',
                    "action_id": "update_data"
                },
                {
                "type": 'dropdown',
                "label": 'Regions',
                "options": [{"label": f"{name}", "value": f"{name}"} for name in regions],
                    "key": 'regions',
                    "action_id": "update_data"
                },
                {
                    "type": 'text',
                    "key": 'year',
                    "lable": 'Years',
                    "value": '1982-2022',
                    "acrion_id": 'simple_html_output'
                },
                {
                    "type": 'text',
                    "key": 'range',
                    "lable": 'data-ranges',
                    "value": '1-48',
                    "acrion_id": 'simple_html_output'
                }
    ]

    tabs = ["Table", "Plot"]

    outputs = [{"output_type":"plot",
                "output_id":"plot_sine_wave",
                "on_page_load":False,
                "control_id": "update_data",
                "tab": "Plot"}, 
                {
                "type": "table",
                "id": "table_id",
                "control_id": "update_data",
                "tab": "Table",
                "on_page_load":False
                }
    ]
    
    controls = [{
        "type": "button",
        "id": "update_data",
        "label": "refresh the page"
    }]
    
    
    def getData(self, params):
        df = create_dataframe('./star')
        return df[(df['area'] == params['regions']) &
                  (df["Week"] <= int(params['range'].split('-')[1])) &
                  (df["Week"] >= int(params['range'].split('-')[0])) &
                  (df["Year"].astype('int64') <= int(params['year'].split('-')[1])) &
                  (df["Year"].astype('int64') >= int(params['year'].split('-')[0]))]

    
    def getPlot(self, params):
        df = create_dataframe('./star')
        df = df[(df['area'] == params['regions']) &
                (df["Week"] <= int(params['range'].split('-')[1])) &
                (df["Week"] >= int(params['range'].split('-')[0]))&
                (df["Year"].astype('int64') <= int(params['year'].split('-')[1])) &
                (df["Year"].astype('int64') >= int(params['year'].split('-')[0]))]
        change_data_format(df)
        sns.set_style("darkgrid")
        plt.figure(figsize=(24,12))
        return sns.scatterplot(x=df['date'], y=df[params['ticker']])
    
    
app = NOAA()
app.launch()

[04/Apr/2022:03:41:18] ENGINE Listening for SIGTERM.
INFO:cherrypy.error:[04/Apr/2022:03:41:18] ENGINE Listening for SIGTERM.
[04/Apr/2022:03:41:18] ENGINE Listening for SIGHUP.
INFO:cherrypy.error:[04/Apr/2022:03:41:18] ENGINE Listening for SIGHUP.
[04/Apr/2022:03:41:18] ENGINE Listening for SIGUSR1.
INFO:cherrypy.error:[04/Apr/2022:03:41:18] ENGINE Listening for SIGUSR1.
[04/Apr/2022:03:41:18] ENGINE Bus STARTING
INFO:cherrypy.error:[04/Apr/2022:03:41:18] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[04/Apr/2022:03:41:18] ENGINE Started monitor thread 'Autoreloader'.
INFO:cherrypy.error:[04/Apr/2022:03:41:18] ENGINE Started monitor thread 'Autoreloader'.
[04/Apr/2022:03:41:18] ENGINE Serving on http://127.0.0.1:8080
INFO:cherrypy.error:[04/Apr/2022:03:41:18] ENGINE Serving on http://127.0.0.1:8080
[04/Apr/2022:03:41:18] ENGINE Bus STARTED
INFO:cherrypy.error:[04/Apr/2022:03:41:18] ENGINE Bus STARTED


127.0.0.1 - - [04/Apr/2022:03:41:33] "GET / HTTP/1.1" 200 442307 "" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
